# Converting database to pandas DataFrame
```Python
with read_engine.connect() as conn:
    conn.execute(text("SET TRANSACTION ISOLATION LEVEL READ COMMITTED"))

    # Read only the filtered points from the database
    points = gpd.read_postgis(
        sql_query,
        conn,
        geom_col='geometry'
    )

# 'points' contains only the data for trajectories with more than 1 point
points_df = pd.DataFrame(points.drop(columns=['geometry', 'descriptors']))
points_df['x'] = points.geometry.x
points_df['y'] = points.geometry.y

points_df.to_feather('points.feather')
```

In [ ]:
from datetime import datetime
from multiprocessing import Pool

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import geopandas as gpd

from pysida.lib import GetSatPairs, get_deformation_from_pair, get_nn_distances

DAYS_IN_SECONDS = 24 * 60 * 60

In [ ]:
# Read feather file and convert to a DataFrame usable by pysida
feather_file = 'points.feather'
#points = pd.read_feather(feather_file)
#df = points[['image_id', 'trajectory_id', 'time', 'x', 'y', 'corr']].copy().rename(columns={'image_id':'i', 'trajectory_id': 'g', 'time':'d', 'x':'x', 'y':'y', 'corr':'q'})
df = pd.read_feather(feather_file)
print(df.shape)
df.head()

In [ ]:
# plot all points between two days
sub_df = df[(df.d > datetime(2020, 1, 10)) & (df.d < datetime(2020, 1, 11))]
plt.scatter(sub_df.x, sub_df.y, c=sub_df.g, s=1)
plt.colorbar()
print(sub_df.shape, sub_df.g.unique().size)

In [ ]:
# plot histogram of trajectory lengths between two dates
sub_df = df[(df.d > datetime(2020, 1, 10)) & (df.d < datetime(2020, 1, 15))]
traj_lengths = [grp.shape[0]  for _, grp in sub_df.groupby('g')]
_ = plt.hist(traj_lengths, bins=30)

In [ ]:
# plot longest trajectories
for _, grp in sub_df.groupby('g'):
    if grp.shape[0] > 20:
        grp_sorted = grp.sort_values('d')        
        plt.plot(grp_sorted.x, grp_sorted.y, '.-')

In [ ]:
# Create a list of Pairs.
# A Pair - contains start and stop points from two images (x0, y0, d0, x1, y1, d1).
# It also contains triangulation, area and perimeter of the faces.

In [ ]:
date_begin = datetime(2020, 1, 10)
date_end = datetime(2020, 1, 15)

# Get pairs for a short period of time
gsp = GetSatPairs(df, min_time_diff = 0.5, max_time_diff = 1.5)
pairs = gsp.get_all_pairs(date_begin, date_end)

In [ ]:
sizes = [pair.x0.size for pair in pairs]
print(len(pairs), min(sizes), max(sizes), pairs[0].__dict__.keys())

In [ ]:
# select the largest pair and plot
pair = pairs[np.argmax(sizes)]
plt.quiver(pair.x0, pair.y0, pair.x1 - pair.x0, pair.y1 - pair.y0, angles='xy', scale_units='xy', scale=1)
print(pair.d0, pair.d1)

In [ ]:
# compute deformation for selected pairs
with Pool(4) as pool:
    defors = pool.map(get_deformation_from_pair, pairs)

In [ ]:
# select the same deformation as the largest pair and plot
print(len(pairs), len(defors), defors[0].__dict__.keys())
defor = defors[np.argmax(sizes)]
plt.figure()
plt.tripcolor(pair.x0, pair.y0, pair.t, defor.e2 * DAYS_IN_SECONDS, cmap='plasma_r', mask=~pair.g, clim=(0,0.5))

In [ ]:
for pair, defor in zip(pairs, defors):
    plt.tripcolor(pair.x0, pair.y0, pair.t, defor.e2 * DAYS_IN_SECONDS, cmap='plasma_r', mask=~pair.g, clim=(0,0.5), alpha=0.1)

In [ ]:
# get sizes and time differences of pairs
sizes = [pair.x0.size for pair in pairs]
time_difs = [(pair.d1 - pair.d0).total_seconds()/ DAYS_IN_SECONDS for pair in pairs]

# plot deformation, drift, histograms of differences between points and deformation
for pair, defor in zip(pairs, defors):
    if pair.d0 == pd.Timestamp('2020-01-13 16:45:16') and pair.d1 == pd.Timestamp('2020-01-14 15:48:02'):
        distances = get_nn_distances(pair)        
        fig, axs = plt.subplots(1, 5, figsize=(18, 4))
        qui0 = axs[0].quiver(pair.x0, pair.y0, pair.x1 - pair.x0, pair.y1 - pair.y0, angles='xy', scale_units='xy', scale=0.1, color='k', width=0.002)
        axs[0].set_aspect('equal')

        trp1 = axs[1].tripcolor(pair.x0, pair.y0, pair.t, defor.e2 * 24 * 60 * 60, clim=[0,0.1], mask=~pair.g, cmap='plasma_r')
        plt.colorbar(trp1, ax=axs[1], shrink=0.5, label='Deformation rate (m/day)')
        axs[1].set_aspect('equal')

        _ = axs[2].hist(distances, bins=50)

        _ = axs[3].hist2d(sizes, time_difs, bins=[50, 50], cmap='jet', cmin=1)

        _ = axs[4].hist(pair.a[pair.g], bins=50, range=[0, 1e8])

        plt.show()
        break        

In [ ]:
# plot all points between two days
sub_df = df[(df.d > datetime(2020, 1, 20)) & (df.d < datetime(2020, 1, 21))]
center_x = 0.5e6
center_y = -0.5e6
dist = np.hypot(sub_df.x - center_x, sub_df.y - center_y) < 1e5
plt.scatter(sub_df.x, sub_df.y, c=dist, s=0.1, alpha=0.1)
plt.colorbar()
print(sub_df.shape, sub_df.g.unique().size)

In [ ]:
end_point_g = sub_df[dist].g.unique()

In [ ]:
sub_df = df[(df.d > datetime(2020, 1, 1)) & (df.d < datetime(2020, 1, 21))]

In [ ]:
filtered_df = sub_df[sub_df['g'].isin(end_point_g)]

In [ ]:
sizes = [g.shape[0] for _, g in filtered_df.groupby('g')]
date_diffs = [(g.d.max() - g.d.min()).total_seconds() / DAYS_IN_SECONDS for _, g in filtered_df.groupby('g')]

In [ ]:
_ = plt.hist(sizes, bins=50)

In [ ]:
_ = plt.hist(date_diffs, bins=50)

In [ ]:
%matplotlib inline
plt.close('all')

In [ ]:
y0s_max = np.max(filtered_df.groupby('g').first().y)
y0s_min = np.min(filtered_df.groupby('g').first().y)
traj_xs = []
traj_ys = []
traj_ds = []
plt.figure()
for _, g in filtered_df.groupby('g'):
    date_diff = (g.d.max() - g.d.min()).total_seconds() / DAYS_IN_SECONDS
    if date_diff > 17:
        g_sorted = g.sort_values('d')
        g_days = (g.d - pd.Timestamp('2020-01-01')).dt.total_seconds() / DAYS_IN_SECONDS
        plt.plot(g_sorted.x - g_sorted.x.iloc[0], g_sorted.y - g_sorted.y.iloc[0], '-', alpha=0.1, c=plt.cm.jet((g_sorted.y.iloc[0]  - y0s_min) / (y0s_max - y0s_min)))
        #plt.plot(g_sorted.x, g_sorted.y, '-', alpha=0.2, c=plt.cm.jet((g_sorted.y.iloc[0]  - y0s_min) / (y0s_max - y0s_min)))
        traj_xs.append(g_sorted.x.values)
        traj_ys.append(g_sorted.y.values)
        traj_ds.append(g_days.values)
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
plt.figure(figsize=(8, 15))
for traj_x, traj_y, traj_d in zip(traj_xs, traj_ys, traj_ds):
    plt.plot(traj_d, traj_y, 'k.-', alpha=0.1)
plt.show()

In [ ]:
x0s = []
y0s = []
x1s = []
y1s = []
for _, g in filtered_df.groupby('g'):
    date_diff = (g.d.max() - g.d.min()).total_seconds() / DAYS_IN_SECONDS
    if date_diff > 17.5:
        #plt.plot(g.x, g.y, '-', alpha=0.1)
        g_sort_d = g.sort_values('d')
        x0 = g_sort_d.x.iloc[0]
        y0 = g_sort_d.y.iloc[0]
        x1 = g_sort_d.x.iloc[-1]
        y1 = g_sort_d.y.iloc[-1]
        x0s.append(x0)
        y0s.append(y0)
        x1s.append(x1)
        y1s.append(y1)

x0s, y0s, x1s, y1s = np.array(x0s), np.array(y0s), np.array(x1s), np.array(y1s)

In [ ]:
plt.quiver(x0s, y0s, x1s - x0s, y1s - y0s, angles='xy', scale_units='xy', scale=1)
plt.gca().set_aspect('equal')

In [ ]:
from pysida.lib import get_triangulation, Pair, get_deformation, thin_poisson
r_min = 0.15
t0, a0, p0, r0 = get_triangulation(x0s, y0s)
g0 = (r0 >= r_min)


In [ ]:
p = Pair(x0=x0s, y0=y0s, x1=x1s, y1=y1s, t=t0, a=a0, p=p0, g=g0, d0=pd.Timestamp('2020-01-01'), d1=pd.Timestamp('2020-01-21'))

In [ ]:
u, v = p.get_speed()
ux, uy, vx, vy = p.get_velocity_gradients(u, v)
e1, e2, e3 = get_deformation(ux, uy, vx, vy)


In [ ]:
plt.tripcolor(x1s, y1s, t0, e1 * DAYS_IN_SECONDS, cmap='coolwarm', clim=(-0.2, 0.2), alpha=0.5, mask=~g0)

In [ ]:
plt.triplot(x0s, y0s, t0, mask=~g0)
plt.triplot(x1s, y1s, t0, mask=~g0)

In [ ]:
keep0 = thin_poisson(x0s, y0s, r=10000)

In [ ]:
x0p = x0s[keep0]
y0p = y0s[keep0]
x1p = x1s[keep0]
y1p = y1s[keep0]
plt.plot(x0s, y0s, 'bo')
plt.plot(x0p, y0p, 'r.')
t0p, a0p, p0p, r0p = get_triangulation(x0p, y0p)
g0p = (r0p >= r_min)


In [ ]:
pp = Pair(x0=x0p, y0=y0p, x1=x1p, y1=y1p, t=t0p, a=a0p, p=p0p, g=g0p, d0=pd.Timestamp('2020-01-01'), d1=pd.Timestamp('2020-01-21'))
up, vp = pp.get_speed()
uxp, uyp, vxp, vyp = pp.get_velocity_gradients(up, vp)
e1p, e2p, e3p = get_deformation(uxp, uyp, vxp, vyp)


In [ ]:
plt.tripcolor(x1p, y1p, t0p, e1p * DAYS_IN_SECONDS, cmap='coolwarm', clim=(-0.1, 0.1), alpha=0.5, mask=~g0p)

In [ ]:
plt.triplot(x0p, y0p, t0p, mask=~g0p)
plt.triplot(x1p, y1p, t0p, mask=~g0p)